In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from datetime import date
from datetime import timedelta

from yahoofinancials import YahooFinancials

from scipy.stats import norm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

import warnings
warnings.filterwarnings('ignore')

In [2]:
end = '2021-09-25'
start = '2016-07-20'

In [3]:
weight_df = pd.read_csv('../data/WeightsDF.csv')
weight_df.set_index('Portfolio', inplace=True)

In [4]:
stocks = np.array(weight_df.columns).tolist()

i1 = 0
i2 = 1
portfolio1 = []
portfolio2 = []
for stock in stocks:
    if weight_df[stock][i1] > 0:
        portfolio1.append(stock)
    if weight_df[stock][i2] > 0:
        portfolio2.append(stock)

In [5]:
portfolio_ticker = []
for stock in stocks:
    portfolio_ticker.append(stock+'.NS')

In [6]:
stocks += ['NIFTY']
portfolio_ticker += ['^NSEI']

In [7]:
for stock_name, stock_ticker in zip(stocks, portfolio_ticker):
    
    yf = YahooFinancials(stock_ticker)
    data = yf.get_historical_price_data(start, end, 'daily')
    globals()[stock_name] = pd.DataFrame(data[stock_ticker]['prices'])
    globals()[stock_name] = globals()[stock_name].drop('date', axis=1).set_index('formatted_date')

    globals()[stock_name]['Date'] = pd.to_datetime(globals()[stock_name].index, format="%Y-%m-%d")
    globals()[stock_name].set_index('Date', drop=False, inplace=True)

    globals()[stock_name] = globals()[stock_name].dropna()

In [8]:
TCS

,high,low,open,close,volume,adjclose,Date
Date,,,,,,,
2016-07-20,1251.250000,1224.074951,1225.000000,1247.474976,2815252.0,1129.042114,2016-07-20
2016-07-21,1259.750000,1232.224976,1244.750000,1253.025024,2527148.0,1134.065430,2016-07-21
2016-07-22,1262.000000,1246.000000,1249.449951,1257.550049,1517216.0,1138.160645,2016-07-22
2016-07-25,1281.599976,1252.900024,1256.000000,1279.474976,1688796.0,1161.004517,2016-07-25
2016-07-26,1284.000000,1266.500000,1279.474976,1276.300049,2311290.0,1158.123901,2016-07-26
...,...,...,...,...,...,...,...
2021-09-20,3871.000000,3807.850098,3809.000000,3823.500000,2002008.0,3823.500000,2021-09-20
2021-09-21,3871.550049,3821.649902,3858.000000,3862.949951,1568129.0,3862.949951,2021-09-21
2021-09-22,3896.850098,3852.449951,3862.500000,3862.149902,1744067.0,3862.149902,2021-09-22


In [9]:
closing_df = pd.DataFrame(index=TCS.index, columns=stocks)
for stock in stocks:
    closing_df[stock] = globals()[stock]['close']

In [10]:
closing_df['formatted_date'] = TCS['Date']

In [11]:
closing_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY,formatted_date
Date,,,,,,,,,,,,,,
2016-07-20,NaN,6674.299805,NaN,412.100006,541.549988,133.850006,1247.474976,NaN,906.900024,NaN,1186.500000,NaN,8565.849609,2016-07-20
2016-07-21,NaN,6867.700195,NaN,409.200012,539.950012,131.800003,1253.025024,NaN,910.049988,NaN,1187.849976,NaN,8510.099609,2016-07-21
2016-07-22,NaN,7078.350098,NaN,409.700012,536.224976,131.500000,1257.550049,NaN,903.200012,NaN,1204.650024,NaN,8541.200195,2016-07-22
2016-07-25,NaN,7126.500000,NaN,412.750000,540.424988,133.449997,1279.474976,NaN,906.950012,NaN,1204.150024,NaN,8635.650391,2016-07-25
2016-07-26,NaN,7164.149902,NaN,414.000000,544.349976,131.149994,1276.300049,NaN,902.700012,NaN,1186.449951,NaN,8590.650391,2016-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-20,3707.899902,20322.599609,8099.850098,2077.350098,1687.849976,857.000000,3823.500000,4168.700195,2800.350098,1392.800049,5110.100098,2482.649902,17396.900391,2021-09-20
2021-09-21,3624.550049,20189.550781,8224.599609,2085.850098,1718.449951,858.900024,3862.949951,4107.450195,2812.449951,1407.550049,5179.450195,2413.500000,17562.000000,2021-09-21
2021-09-22,3671.300049,19925.199219,8614.000000,2081.050049,1716.900024,854.150024,3862.149902,4163.149902,2784.500000,1378.699951,5139.899902,2401.949951,17546.650391,2021-09-22


In [12]:
import json
from prophet import Prophet
from prophet.serialize import model_from_json

In [13]:
with open('serialized_model.json', 'r') as fin:
    model = model_from_json(json.load(fin))  

In [14]:
pred_df = pd.DataFrame(columns=stocks)

In [15]:
for stock_name in stocks:
    
    df = closing_df[['formatted_date', stock_name]].copy()
    df.reset_index(drop=True, inplace=True)

    df.rename(columns={"formatted_date": "ds", stock_name: "y"}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'], errors='coerce')
    
    model=Prophet(changepoint_prior_scale= 0.4, n_changepoints=100, seasonality_mode='multiplicative') 
    model.fit(df)

    future = model.make_future_dataframe(periods=90, freq='B')
    forecast = model.predict(future)
    
    pred_df[stock_name] = forecast['yhat']
    pred_df['date'] = forecast['ds']

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this

In [16]:
pred_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY,date
0,-7807.453044,7097.080065,12480.494288,414.380524,542.719769,135.242672,1290.112806,289.061757,920.049943,-248.901999,1153.873551,1611.146917,8638.817006,2016-07-20
1,-7792.243127,7085.181813,12648.320624,413.757322,541.552936,135.214330,1289.021276,291.012305,916.043026,-247.369051,1158.260570,1604.993066,8630.597210,2016-07-21
2,-7759.164037,7068.480094,12715.358428,412.846652,541.282325,135.205638,1287.767408,290.584649,913.813176,-245.283614,1165.630775,1599.288086,8624.084576,2016-07-22
3,-7722.152808,7010.621594,12847.115957,411.531414,539.935877,134.469459,1291.091940,286.536689,906.733760,-239.827004,1176.622012,1569.101600,8593.427515,2016-07-25
4,-7768.296711,6997.349736,12927.522386,412.834677,539.604318,134.683298,1294.178492,286.000146,906.235549,-240.620206,1180.923416,1562.476435,8604.054347,2016-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,4716.765208,21884.765677,448.463227,2411.171093,2123.757978,892.370364,4855.249455,5594.569344,3025.274012,1834.136361,5424.902182,2287.981791,21044.391333,2022-01-24
1364,4761.449999,21840.733093,419.610550,2413.440454,2117.433329,890.688445,4859.179381,5620.275179,3019.011032,1838.109520,5418.189807,2296.133658,21091.003264,2022-01-25
1365,4770.815222,21783.667374,391.227449,2405.731364,2113.156571,887.417918,4855.911958,5641.637064,3010.793626,1841.668051,5412.294458,2305.706233,21109.896415,2022-01-26
1366,4777.996385,21742.851211,366.909113,2403.164368,2103.271715,885.607506,4847.946606,5729.940249,2996.297199,1821.860436,5403.777188,2319.546192,21120.877401,2022-01-27


In [17]:
pred = pred_df[-90:]
pred.set_index('date', drop=True, inplace=True)

In [18]:
pred

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY
date,,,,,,,,,,,,,
2021-09-27,3291.023742,19947.001288,7733.741393,2092.839561,1784.232272,832.383093,4032.361114,4057.256404,2803.330264,1425.250068,5075.305033,2288.307098,17589.119292
2021-09-28,3325.848177,19947.814367,7584.800056,2098.796460,1794.200749,829.079181,4055.618940,4029.646963,2807.413881,1450.010621,5067.411824,2256.266655,17620.569360
2021-09-29,3336.025003,19940.252996,7434.110468,2095.688145,1806.452748,824.049007,4074.229479,3993.771305,2809.969744,1474.711338,5058.651979,2224.321684,17626.292593
2021-09-30,3344.668862,19951.680429,7321.532030,2096.842754,1814.086947,819.989706,4090.243464,4010.319500,2806.938982,1482.023075,5046.102276,2195.082480,17624.141694
2021-10-01,3345.576972,19953.465342,7170.647908,2095.933365,1824.354451,815.930347,4104.802518,3988.765515,2808.517230,1488.171535,5048.205139,2166.230208,17625.472903
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,4716.765208,21884.765677,448.463227,2411.171093,2123.757978,892.370364,4855.249455,5594.569344,3025.274012,1834.136361,5424.902182,2287.981791,21044.391333
2022-01-25,4761.449999,21840.733093,419.610550,2413.440454,2117.433329,890.688445,4859.179381,5620.275179,3019.011032,1838.109520,5418.189807,2296.133658,21091.003264
2022-01-26,4770.815222,21783.667374,391.227449,2405.731364,2113.156571,887.417918,4855.911958,5641.637064,3010.793626,1841.668051,5412.294458,2305.706233,21109.896415


In [19]:
monthly_return = pred.resample('M').ffill().pct_change()
monthly_return

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY
date,,,,,,,,,,,,,
2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-31,0.100770,0.049286,-0.397933,0.009815,0.037537,-0.052628,0.028791,0.055100,0.006691,0.074670,-0.010048,-0.063475,0.033386
2021-11-30,0.109775,0.048879,-0.438519,0.089021,0.003344,0.090184,-0.018309,0.140149,0.015940,0.396309,0.122678,-0.057695,0.075423
2021-12-31,0.086185,0.045935,-0.491723,0.098787,0.090006,0.086811,0.092414,0.170625,0.086219,-0.099401,0.043603,0.024847,0.051483
2022-01-31,0.075740,-0.055694,-0.730688,-0.053393,0.018367,-0.039373,0.071813,0.022952,-0.042224,-0.100536,-0.075106,0.175844,0.026316


In [20]:
monthly_closing_df = pred.resample('M').mean()
monthly_closing_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY
date,,,,,,,,,,,,,
2021-09-30,3324.391446,19946.687270,7518.545987,2096.041730,1799.743179,826.375247,4063.113249,4022.748543,2806.913218,1457.998775,5061.867778,2240.994479,17615.030735
2021-10-31,3529.940750,20343.220283,5536.233959,2113.374085,1887.243459,776.530142,4229.405799,4059.197913,2824.597348,1569.825497,4980.466454,1998.954614,17851.757718
2021-11-30,3893.265994,21484.535032,3261.719158,2182.367225,1873.373558,803.885768,4134.933391,4357.444790,2805.771429,1955.039497,5311.933389,1951.148889,18918.476613
2021-12-31,4271.406767,22785.657366,1729.659708,2438.771245,1960.690644,889.909248,4305.446164,5152.904694,3033.477750,2127.994177,5746.289490,1951.174089,20101.524903
2022-01-31,4633.916952,22278.524179,737.646115,2466.584086,2124.147216,913.542426,4757.102624,5685.071579,3076.767339,1927.136629,5592.764254,2198.607067,20946.281361


In [21]:
nifty_std = monthly_return['NIFTY'].std()

mstd = monthly_return.copy()
mstdl = mstd.corr(method='pearson')

In [22]:
monthly_stock_beta = pd.Series(index=stocks)

for stock in stocks:
    monthly_stock_beta[stock] = (mstdl['NIFTY'][stock] * mstd[stock].std()) / nifty_std

In [23]:
monthly_stock_beta

IRCTC         0.484253
NESTLEIND     1.462507
INDIAMART     3.572011
TITAN         2.772379
INFY         -0.236324
TATACONSUM    3.132562
TCS          -1.334554
DIXON         2.539680
HINDUNILVR    1.141845
ATGL          8.312860
DIVISLAB      3.748908
POLYCAB      -2.966539
NIFTY         1.000000
dtype: float64

In [24]:
w1 = weight_df.iloc[0].dropna()
w2 = weight_df.iloc[1].dropna()

In [25]:
metrics = ['Return','Volatility','Sharpe Ratio', 'VaR', 'Portfolio Beta', 'Treynor Ratio', 'Jensen Alpha']
metrics_df = pd.DataFrame(columns=metrics)

In [26]:
nifty_50_daily = pred['NIFTY'].pct_change().dropna()

In [27]:
risk_free_rate = 0.0602
alpha = 0.01
days = 21
market_return = nifty_50_daily.mean() * days 

In [28]:
# P1

r1 = np.zeros(len(metrics))
sb1 = monthly_stock_beta[portfolio1].copy()
pb1 = 0
tdf1 = monthly_return[portfolio1].copy()
twl1 = w1
tcovdf1 = tdf1.cov()

pr1 = np.sum(tdf1.mean() * twl1) * days
pstd1 = np.sqrt(np.dot(twl1.T,np.dot(tcovdf1, twl1))) * np.sqrt(days)

r1[0] = pr1
r1[1] = pstd1

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r1[2] = (r1[0] - risk_free_rate) / r1[1]

# VaR => return - (Z * volatility)
r1[3] = abs(pr1 - (pstd1 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl1)):
    pb1 += twl1[j] * sb1[j]
r1[4] = pb1

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r1[5] = (r1[0] - risk_free_rate) / r1[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r1[6] = r1[0] - (risk_free_rate + r1[4] * (market_return - risk_free_rate))   

r1 = pd.Series(r1, index=metrics)
metrics_df = metrics_df.append(r1, ignore_index=True)

In [29]:
# P2

r2 = np.zeros(len(metrics))
sb2 = monthly_stock_beta[portfolio2].copy()
pb2 = 0
tdf2 = monthly_return[portfolio2].copy()
twl2 = w2
tcovdf2 = tdf2.cov()

pr2 = np.sum(tdf2.mean() * twl2) * days
pstd2 = np.sqrt(np.dot(twl2.T,np.dot(tcovdf2, twl2))) * np.sqrt(days)

r2[0] = pr2
r2[1] = pstd2

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r2[2] = (r2[0] - risk_free_rate) / r2[1]

# VaR => return - (Z * volatility)
r2[3] = abs(pr2 - (pstd2 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl2)):
    pb2 += twl2[j] * sb2[j]
r2[4] = pb2

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r2[5] = (r2[0] - risk_free_rate) / r2[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r2[6] = r2[0] - (risk_free_rate + r2[4] * (market_return - risk_free_rate))   

r2 = pd.Series(r2, index=metrics)
metrics_df = metrics_df.append(r2, ignore_index=True)

In [30]:
metrics_df

,Return,Volatility,Sharpe Ratio,VaR,Portfolio Beta,Treynor Ratio,Jensen Alpha
0,-2.658421,0.504180,-5.392163,3.831319,3.880706,-0.700548,-2.653474
1,-2.229704,0.409994,-5.585209,3.183494,3.261310,-0.702142,-2.235155
